In [58]:
from typing import cast
from itertools import combinations
import scipy.optimize as opt
from pyscf import gto
from qiskit import QuantumCircuit
from qiskit_aer import AerProvider
from qc_practice import VQE, SSVQE
from qc_practice.ansatz import UCCSD, UpCCGSD, HEA, SP, RSP

In [102]:
from networkx import identified_nodes
from sympy import symbols, Matrix, cos, sin, init_printing, pi, exp, I
from sympy.tensor.array import tensorproduct
from sympy.physics.quantum import Dagger

init_printing(use_unicode=True)

theta, phi = symbols('theta phi')
Z = Matrix([[1, 0], [0, -1]])
Y = Matrix([[0, -1j], [1j, 0]])

Ry = Matrix([
    [cos(theta/2), -sin(theta/2)],
    [sin(theta/2), cos(theta/2)]
])

Rz = Matrix([
    [exp(I * (phi) / 2), 0],
    [0, exp(-I * (phi) / 2)]
])

Rz_dagger = Matrix([
    [exp(-I * (phi) / 2), 0],
    [0, exp(I * (phi) / 2)]
])

Identity = Matrix([
    [1, 0],
    [0, 1]
])

RzRy = Rz * Ry

RzRy_dagger = Ry * Rz_dagger


CNOT = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

CNOT2 = Matrix([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

def tensor(A, B):
    C = tensorproduct(A, B)

    D = Matrix([
        [C[0][0][0][0], C[0][0][0][1], C[0][1][0][0], C[0][1][0][1]],
        [C[0][0][1][0], C[0][0][1][1], C[0][1][1][0], C[0][1][1][1]],
        [C[1][0][0][0], C[1][0][0][1], C[1][1][0][0], C[1][1][0][1]],
        [C[1][0][1][0], C[1][0][1][1], C[1][1][1][0], C[1][1][1][1]]
    ])

    return D

Rgate = tensor(Identity, RzRy)
Rgate_dagger = tensor(Identity, RzRy_dagger)

M1 = Rgate_dagger  * CNOT2

M2 = CNOT * M1

M3 = Rgate * M2

M3

M4 = CNOT2 * M3

M4


⎡         ⅈ⋅φ    2⎛θ⎞    -ⅈ⋅φ    2⎛θ⎞         ⅈ⋅φ    ⎛θ⎞    ⎛θ⎞    -ⅈ⋅φ    ⎛θ⎞
⎢      - ℯ   ⋅sin ⎜─⎟ + ℯ    ⋅cos ⎜─⎟        ℯ   ⋅sin⎜─⎟⋅cos⎜─⎟ + ℯ    ⋅sin⎜─⎟
⎢                 ⎝2⎠             ⎝2⎠                ⎝2⎠    ⎝2⎠            ⎝2⎠
⎢                                                                             
⎢   ⅈ⋅φ    ⎛θ⎞    ⎛θ⎞    -ⅈ⋅φ    ⎛θ⎞    ⎛θ⎞         ⅈ⋅φ    2⎛θ⎞    -ⅈ⋅φ    2⎛θ
⎢- ℯ   ⋅sin⎜─⎟⋅cos⎜─⎟ - ℯ    ⋅sin⎜─⎟⋅cos⎜─⎟        ℯ   ⋅cos ⎜─⎟ - ℯ    ⋅sin ⎜─
⎢          ⎝2⎠    ⎝2⎠            ⎝2⎠    ⎝2⎠                 ⎝2⎠             ⎝2
⎢                                                                             
⎢                                                                             
⎢                    0                                          0             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [2]:
mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
vqe = SSVQE(mol)
vqe.ansatz = RSP(depth=7)
vqe.parallel = True
vqe.run()

Computing Hamiltonian...... Done
SCF Electronic Energy: -1.873316479161709
SCF Total Energy:      -1.117349034990280


Starting SSVQE Calculation

Ansatz: RSP
Optimizer: powell
Active Space: [1, 1]
Koopmans Condition: False
Weights: [10, 5, 4, 3, 2, 1]

Iteration: 1
State_0 Energy: -1.117707036142402
(0, 1)
State_1 Energy: -0.299577780181568
(0, 3)
State_2 Energy: -0.479007957625523
(2, 1)
State_3 Energy: -0.477978704313170
(2, 3)
State_4 Energy: -0.298834927790914
((0, 2), (1, 3))
State_5 Energy:  0.565431337222866

Iteration: 2
State_0 Energy: -0.741183169111423
(0, 1)
State_1 Energy: -0.675690079032909
(0, 3)
State_2 Energy: -0.477978704313170
(2, 1)
State_3 Energy: -0.477710203449078
(2, 3)
State_4 Energy:  0.100388880558987
((0, 2), (1, 3))
State_5 Energy:  0.176312225739338

Iteration: 3
State_0 Energy: -0.743036592260238
(0, 1)
State_1 Energy: -0.673122101130926
(0, 3)
State_2 Energy: -0.477683353362668
(2, 1)
State_3 Energy: -0.478649956473399
(2, 3)
State_4 Energy:  0.09773879

[Singlet_0, Singlet_1, Triplet_2, Triplet_3, Singlet_4, Singlet_5]

1. run VQE

In [2]:
mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
vqe = VQE(mol)
vqe.ansatz = UpCCGSD()
vqe.parallel = True
vqe.run()

Computing Hamiltonian...... Done
SCF Electronic Energy: -1.873316479161709
SCF Total Energy:      -1.117349034990280


Starting SSVQE Calculation

Ansatz: SP
Optimizer: powell
Active Space: [1, 1]
Koopmans Condition: False
Weights: [10, 5, 4, 3, 2, 1]

Iteration: 1
State_0 Energy: -1.117053684039778
(0, 1)
State_1 Energy: -0.299183978914233
(0, 3)
State_2 Energy: -0.477540152901819
(2, 1)
State_3 Energy: -0.479276458489615
(2, 3)
State_4 Energy: -0.298127875515471
((0, 2), (1, 3))
State_5 Energy:  0.563498131001403

Iteration: 2
State_0 Energy: -1.119031640405256
(0, 1)
State_1 Energy: -0.299837331016858
(0, 3)
State_2 Energy: -0.478954257452704
(2, 1)
State_3 Energy: -0.478614156358187
(2, 3)
State_4 Energy: -0.298763327560489
((0, 2), (1, 3))
State_5 Energy:  0.565878838663019

Iteration: 3
State_0 Energy: -0.534111769704707
(0, 1)
State_1 Energy: -0.882479518574983
(0, 3)
State_2 Energy: -0.479142208057568
(2, 1)
State_3 Energy: -0.478426205753323
(2, 3)
State_4 Energy:  0.314443743

[Singlet_0, Singlet_1, Triplet_2, Triplet_3, Singlet_4, Singlet_5]

2. set qc[0] as the ground state

In [5]:
number_of_states = 2

no = vqe.profile.num_orb
qc = [QuantumCircuit(no*2, no*2) for _ in range(number_of_states)]
ancila = QuantumCircuit(1, 1)

vqe._initialize_circuit(qc[0])
vqe.ansatz.ansatz(qc[0], vqe.profile, vqe.profile.coeff)


coeff = vqe.ansatz.generate_coeff(vqe.profile)
vqe._initialize_circuit(qc[1])
vqe.ansatz.ansatz(qc[1], vqe.profile, coeff)

3. measuring overlap

In [7]:
def measure_overlap(qc1, qc2):
    no = vqe.profile.num_orb
    ancila = QuantumCircuit(1, 1)
    qc_total = ancila.tensor(qc1).tensor(qc2)
    qc_total.h(0)
    for i in range(1, no*2+1):
        qc_total.ccx(i+no*2, i, 0)
    qc_total.h(0)

    backend = AerProvider().get_backend('qasm_simulator')
    result = cast(dict[str, float],
                backend.run(qc_total, shots=10000).result().get_counts())
    print(result)

measure_overlap(qc[0], qc[1])

/tmp/ipykernel_12656/6180801.py:10: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = AerProvider().get_backend('qasm_simulator')


QiskitError: 'No counts for experiment "0"'

4. make cost function

4. run optimizer